In [2]:
pip install qdrant_client

   ---------------------------------------- 0.0/206.3 kB ? eta -:--:--
   ------------------------------------- -- 194.6/206.3 kB 5.8 MB/s eta 0:00:01
   ---------------------------------------- 206.3/206.3 kB 2.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   -- ------------------------------------- 0.2/3.8 MB ? eta -:--:--
   ------ --------------------------------- 0.6/3.8 MB 8.1 MB/s eta 0:00:01
   --------------- ------------------------ 1.4/3.8 MB 11.5 MB/s eta 0:00:01
   ------------------ --------------------- 1.7/3.8 MB 10.1 MB/s eta 0:00:01
   ------------------------- -------------- 2.4/3.8 MB 10.9 MB/s eta 0:00:01
   --------------------------------- ------ 3.2/3.8 MB 11.9 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 12.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------- ----- 0.9/1.1 MB 29.7 MB/s eta 0:00:01
   ---------------------

In [4]:
pip install sentence_transformers

     ---------------------------------------- 0.0/130.7 kB ? eta -:--:--
     --- ------------------------------------ 10.2/130.7 kB ? eta -:--:--
     ----------------------------------- -- 122.9/130.7 kB 2.4 MB/s eta 0:00:01
     -------------------------------------- 130.7/130.7 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/156.5 kB ? eta -:--:--
   ---------------------------------------- 156.5/156.5 kB 4.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/346.2 kB ? eta -:--:--
   --------------------------------------  337.9/346.2 kB 10.6 MB/s eta 0:00:01
   ---------------------------------------- 346.2/346.2 kB 7.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/198.6 MB ? eta -:--:--
   ---------------------------------------- 0.7/198.6 MB 21.1 MB/s eta 0:00:10
   ---------------------------------------- 1.6/198.6 MB 20.2 MB/s eta 0:00:10
   ---------------------------------------- 2.4/198.6 MB 19.1 MB/s eta 0:00:11
   

In [5]:
import pandas as pd
import requests
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/alfredodeza/learn-retrieval-augmented-generation/main/top_rated_wines.csv')

df.dropna(inplace=True)

data = df.sample(100).to_dict('records')

In [7]:
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [19]:
encoder = SentenceTransformer('all-MiniLM-L6-v2')

In [20]:
encoder

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [9]:
# Create collection to store wines data
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE
    )
)

True

In [11]:
# Vectorize!
qdrant.upload_points(
    collection_name="top_wines",
    points = [
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc['notes']).tolist(), # query({"inputs": [doc['description']]}),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable / dict holding all the wines
    ]
)

In [12]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

In [13]:
# Search / Retrieve
hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode(user_prompt).tolist(), # query({"inputs": [user_prompt]}), #Query Text
    limit=3
)
for hit in hits:
    print(hit.payload, "score: ", hit.score)

search_results = [hit.payload for hit in hits]

{'name': 'Bodega Colome Altura Maxima Malbec 2012', 'region': 'Salta, Argentina', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'Winemaker Thibaut Delmotte has crafted wines of distinction and international acclaim for Colome. He believes the Malbec from Altura Maxima Vineyard is the embodiment of two extremes - a traditional grape variety from his French origins made from the vineyard that challenges all convention in the modern viticultural world.'} score:  0.617968136831349
{'name': "Brovia Ca'Mia Barolo (1.5 Liter Magnum) 2006", 'region': 'Barolo, Piedmont, Italy', 'variety': 'Red Wine', 'rating': 96.0, 'notes': "A very complete and long aging wine from a great vineyard of Serralunga d'Alba. The color is intense ruby red. The smell is intense, pleasant, balanced and spicy with a bouquet of plums, cedar, tobacco, liquorice, soy and floral notes. The taste is full bodied, concentrated and rich, with a firm tannic structure that classic Barolo needs for a long aging."} score:  0.578

In [15]:
pip install openai


   ---------------------------------------- 0.0/227.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/227.4 kB ? eta -:--:--
   ----- ---------------------------------- 30.7/227.4 kB 1.3 MB/s eta 0:00:01
   -------------- ------------------------ 81.9/227.4 kB 919.0 kB/s eta 0:00:01
   ---------------------------------------  225.3/227.4 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 227.4/227.4 kB 1.5 MB/s eta 0:00:00


In [16]:
# Now time to connect to the local large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="LLaMA_CPP", # Local model stored as LAMAFILE
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="[{'name': 'Bodega Colome Altura Maxima Malbec 2012', 'region': 'Salta, Argentina', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'Winemaker Thibaut Delmotte has crafted wines of distinction and international acclaim for Colome. He believes the Malbec from Altura Maxima Vineyard is the embodiment of two extremes - a traditional grape variety from his French origins made from the vineyard that challenges all convention in the modern viticultural world.'}]", role='assistant', function_call=None, tool_calls=None)


In [17]:
# Now time to connect to the local large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8081/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="LLaMA_CPP", # Local model stored as LAMAFILE
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="I suggest you to try 'Brovia Ca'Mia Barolo (1.5 Liter Magnum) 2006', it is a great wine from Italy.", role='assistant', function_call=None, tool_calls=None)
